## Instalando librerías para el web scrapping

In [3]:
!pip install requests
!pip install bs4
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib

You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.

You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.


## Importando librerías

In [4]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import bs4

###  URL de los recursos

In [5]:
baseurl ='https://statsroyale.com/es'

In [6]:
headers ={'User-Agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 10_0 like Mac OS X) AppleWebKit/602.4.6 (KHTML, like Gecko) Version/10.0 Mobile/14A346 Safari/E7FBAF'}

In [7]:
r = requests.get('https://statsroyale.com/es/cards')

In [8]:
soup = BeautifulSoup(r.content, 'lxml')

In [9]:
cardlist =soup.find_all('div',class_="cards__card")

Guardando Links

In [10]:
cardlinks =[]
for item in cardlist:
    for link in item.find_all('a', href= True): 
        cardlinks.append(link['href'])

Verificando la cantidad de Links extraídos

In [11]:
print('Numero de links:',len(cardlinks))

Numero de links: 102


## Extrayendo información de cada carta 

In [12]:
lista_diccionario = []

In [13]:
!pip install regex

You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.


### Patrones para clasificar las cartas.

In [14]:
import re
#Patrones 
      #GOLEN Y GIGANTES
patron_gigante = re.compile(r'\b([G|g]\w+te)|([G|g][o|ó]\w+m)\b')
      #muerto
patron_muerto = re.compile(r'(\b[E|e]sq\w+[o|s])|([R|L][o|á]\w+[s|a])\b')
      #CRIATURAS O CARTAS HIELO
patron_criatura =re.compile(r'\b(([H|h]i\w+o)|([n|F][i|a]\w+[a|e])|(E\w+\W*\w+[o|u]s)|(Mu\w+\W*\w+os))\b')
      #CARTAS DRAGON
patron_dragon = re.compile(r'\b([D|d]r\w+\W*[n|s])\b')
      #CARTAS DE DUENDES
patron_duende = re.compile(r'\b([D|d]u\w+[e|s])\b')
      #CARTAS DE MAGOS Y BRUJAS
patron_magico = re.compile(r'\b((Ma\w+o)|(Br\w+a))\b')
      #CARTAS DE ESTRUCTURAS (CHOZAS)
patron_choza = re.compile(r'\b((Ch\w+a))\b')
      # CARTAS DE ESTRUCTURAS TORRES
patron_torre = re.compile(r'\b((To\w+e))\b')
      #CARTAS GUERREROS HUMANOS
patron_humano = re.compile(r'\b((P[r|e]\W*\w+[a|e|r])|(Ca\w+[r|o])|([M|g|B][a|u|e|i]\w+[e|i][d|r][o|a])|([V|M|m]\w+as*)|(P[r|e]\W*\w+[a|e|r])|(Ca\w+[r|o])|([M|g|B][a|u|e|i]\w+[e|i][d|r][o|a])|([V|M|m]\w+as*))\b')

### Lista de atributos que se extraeran:
    Nombre                             name
    elixir                             elixir
    Nivel                              level
    Calidad                            quality
    Objetivo                           objective
    Alcance                            scope
    Velocidad                          speed
    Radio                              radio
    Tiempo de vida                     time of life
    Puntos de vida                     Life points
    Daño                               Hurt
    Daño\s                             Damage
    Daño de área                       Area damage

In [15]:
lista_elixir = []
i=0
for item in soup.find_all('div',class_="cards__card"):
    lista_elixir.append(item['data-elixir'])

for link in cardlinks:
    r = requests.get(link, headers = headers)
    soup = BeautifulSoup(r.content,'lxml')
    #####################################################
    #Creamos variables de aquella informacion que no tiene una clase ni etiqueta que los distinga 
    # ademas que puede o no que existan estas variable en cada uno de los links
    lifepoints = 'None'
    hurt = 'None'
    damage = 'None'
    areadamage = 'None'
    leve = soup.find('tbody')
    
    try:
        #recorremos todos los valores de etiqueta con la clase CARD_TABLEVALUES
        for item in leve.find_all('tr',class_='card__tableValues'):
            if 'Puntos de vida'== item.find('strong').text.strip():
                lifepoints = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño'== item.find('strong').text.strip():
                hurt = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño/s'== item.find('strong').text.strip():
                damage = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            if 'Daño de área' == item.find('strong').text.strip():
                areadamage = item.find('td',class_ ='card__tableValue card__isActive').text.strip()
            #print(type(item.find('strong').text.strip()))
            #print(item.find('strong').text.strip(), item.find('td',class_ ='card__tableValue card__isActive').text.strip())
            #print(item.find('td',class_ ='card__tableValue card__isActive').text)
    except:
        lifepoints = 'None'
        hurt = 'None'
        damage = 'None'
        areadamage = 'None'
####################################################################

    name =soup.find('div',class_='ui__headerMedium card__cardName').text.strip()
    #############################################################################################
    classes = []
    if patron_gigante.search(name):
        classes.append('gigante')
    if patron_muerto.search(name):
        classes.append('muerto')
    if patron_criatura.search(name):
        classes.append('criatura')
    if patron_dragon.search(name):
        classes.append('dragon')
    if patron_duende.search(name):
        classes.append('duende')
    if patron_magico.search(name):
        classes.append('magico')
    if patron_choza.search(name):
        classes.append('choza')
    if patron_torre.search(name):
        classes.append('torre')
    if patron_humano.search(name):
        classes.append('humano')
    else:
        classes.append('otro')
    #############################################################################################
    try:
        level = soup.find('th', class_='card__tableHeader card__isActiveHeader').text.strip()
    except:
        level = 'None'
    try:
        quality = soup.find('span',class_='card__rare').text.strip()
    except:
        try:
            quality = soup.find('span',class_='card__legendary').text.strip()
        except:
            try:
                quality = soup.find('span',class_='card__epic').text.strip()
            except:
                try:
                    quality = soup.find('span',class_='card__common').text.strip()
                except:
                    quality = 'None'
    try:
        objective = soup.find('div', class_='card__metric card__target').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        objective = 'None'
    try:
        scope = soup.find('div',class_='card__metric card__range').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        scope = 'None'
    try:
        speed = soup.find('div',class_ ='card__metric card__movementSpeed').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        speed = 'None'
    try:
        radio = soup.find('div',class_='card__metric card__radius').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        radio = 'None'
    try:
        timeoflife = soup.find('div',class_ = 'card__metric card__durationLifetime').find('div',class_='ui__mediumText card__count').text.strip()
    except:
        timeoflife = 'None'
#Creamos la estructura del diccionario
    CARDS ={
        'name':name,
        'level':level,
        'quality':quality,
        'objective': objective,
        'scope':scope,
        'speed':speed,
        'radio': radio,
        'timeoflife':timeoflife,
        ######################################
        'lifepoints' : lifepoints,
        'hurt' : hurt,
        'damage' : damage,
        'areadamage' : areadamage,
        'link':link,
        'classes': classes,
        'elixir' : lista_elixir[i]
    }
    #guardamos los diccionarios en una lista
    lista_diccionario.append(CARDS)
    i=i+1

Verificando la cantidad de diccionarios en la lista

In [16]:
# Número de Cartas
print(len(lista_diccionario))

102


# Modelando Grafo

In [17]:
# install rdflib 
!pip3 install rdflib

You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.


In [18]:
# Nodes 
from rdflib import URIRef, BNode, Literal
# Namespace 
from rdflib import Namespace
# resources
from rdflib.namespace import RDF, RDFS
# Graph 
from rdflib import ConjunctiveGraph
import pprint

## Clash Royale
[Guia completa](https://www.rtve.es/playz/clash-royale/como-jugar/)

Clash Royale es el juego de lucha de ritmo rápido en el que recolectas cartas y luchas contra otros jugadores en tiempo real. Defiende tus torres y destruye las de tu enemigo con tus mejores estrategias de ataque y defensa para ganar la partida.

#### ¿Qué tipos de cartas existen?
Existen cuatro tipos de cartas dependiendo de su poder que podrás ir desbloqueando según avances en el juego: **comunes**, **especiales**, **épicas** y **legendarias**. Cada una de ellas tiene un **nivel de poder**. Para invocar una carta en la pelea tendrás que tener en cuenta el costo de elixir que consume, que será mayor cuanto más poder tenga. ¡Cuidado! si usas muchas cartas con un consumo de elixir muy elevado, será muy dificil invocar varias cartas seguidas.

#### ¿Cómo se lucha?
En las batallas, los dos adversarios disponen de sus respectivos mazos en **rotaciones de cuatro**. Según utilices las cartas, irán apareciendo otras nuevas para que las sigas combinando a tu criterio. Cada jugador tiene una barra de **diez unidades de elixir** que se va a recargando progresivamente. 

#### ¿Cuál es el objetivo de las batallas?
En la arena tu campo es la **mitad inferior** y el del oponente, la **mitad superior**. Tus tropas tendrán que alcanzar las **torres del enemigo** para derribarlas a toda costa antes de que se acaben los **tres minutos de tiempo**. Para ello tendrás que derrotar a los enemigos que el rival invoque al campo de batalla con el objetivo de defenderte o atacar sus tropas. 

## Cartas


### Clases de cartas:
1. Tropa
2. Estructura
3. Hechizo

#### Clases en tropas
1. Duendes
2. Gigantes
3. Dragones
4. Muertos
5. Criaturas
6. Mágico
7. Humanos

#### Clases en estructuras

1. Chozas
2. Torres


#### Atributos de las cartas
- Costo de elixir
- Nombre de la carta             
- Nivel                           
- Calidad de carta                       
- Objetivo                      
- Velocidad                                  
- Radio 
- Tiempo de vida                  
- Puntos de vida
- Daño
- Daño\s   
- Daño de área

#### Clases de calidad de la carta
1. Especial
2. Épica
3. Común
4. Legendaria

#### Clases de objetivos de la carta
1. Tierra
2. Estructura
3. Aire

#### Clases de alcance de las cartas
1. Cuerpo a cuerpo
2. Cerca
3. Lejos

#### Clases de velocidades de las cartas
1. Lento
2. Media
3. Rápido
4. Muy rápido




In [19]:
n = Namespace("https://statsroyale.com/es/")

In [85]:
# Creando las URIRef
# Propiedades de la clase Característica
name = n.name
level = n.level
quality = n.quality
objective = n.objective
scope = n.scope
speed = n.speed
radio = n.radio
timeoflife = n.timeoflife
lifepoints = n.lifepoints
hurt = n.hurt
damage = n.damage
areadamage = n.areadamage
elixir = n.elixir
# Subpropiedades de calidad
rare = n.rare
epic = n.epic
common = n.common
legendary = n.legendary

# Subpropiedades de objetivo
air = n.air
building = n.building
ground = n.ground

# Subpropiedades de alcance
melee = n.melee
near = n.near
far = n.far

# Subpropiedades de velocidad
slow = n.slow
medium = n.medium
fast = n.fast
veryfast = n.veryfast


In [21]:
g = ConjunctiveGraph()
print(g)

[a rdflib:ConjunctiveGraph;rdflib:storage [a rdflib:Store;rdfs:label 'IOMemory']]


In [22]:
# Añadiendo las subclases al grafo
# subclases de clase maestra: Objeto carta y subclases sucesoras
g.add((n.troop, RDFS.subClassOf, n.card))
g.add((n.build, RDFS.subClassOf, n.card))
g.add((n.spell, RDFS.subClassOf, n.card))

# subclases de la carta tropa
g.add([n.giant, RDFS.subClassOf, n.troop])
g.add([n.dead, RDFS.subClassOf, n.troop])
g.add([n.creature, RDFS.subClassOf, n.troop])
g.add([n.dragon, RDFS.subClassOf, n.troop])
g.add([n.goblin, RDFS.subClassOf, n.troop])
g.add([n.human, RDFS.subClassOf, n.troop])
g.add([n.magic, RDFS.subClassOf, n.troop])

# subclases de la carta estructura
g.add([n.hut, RDFS.subClassOf, n.build])
g.add([n.tower, RDFS.subClassOf, n.build])


In [23]:
# Añadiendo las subpropiedades al grafo
# Subpropiedades de la clase Característica
g.add([name, RDFS.subPropertyOf, n.characteristic])
g.add([level, RDFS.subPropertyOf, n.characteristic])
g.add([quality, RDFS.subPropertyOf, n.characteristic])
g.add([objective, RDFS.subPropertyOf, n.characteristic])
g.add([scope, RDFS.subPropertyOf, n.characteristic])
g.add([speed, RDFS.subPropertyOf, n.characteristic])
g.add([radio, RDFS.subPropertyOf, n.characteristic])
g.add([timeoflife, RDFS.subPropertyOf, n.characteristic])
g.add([lifepoints, RDFS.subPropertyOf, n.characteristic])
g.add([hurt, RDFS.subPropertyOf, n.characteristic])
g.add([damage, RDFS.subPropertyOf, n.characteristic])
g.add([areadamage, RDFS.subPropertyOf, n.characteristic])
g.add([elixir, RDFS.subPropertyOf, n.characteristic])
# Subpropiedades de calidad
g.add([rare ,RDF.type, quality ])
g.add([epic ,RDF.type, quality ])
g.add([common ,RDF.type, quality ])
g.add([legendary ,RDF.type, quality ])

# Subpropiedades de objetivo
g.add([air , RDF.type, objective ])
g.add([building , RDF.type, objective ])
g.add([ground , RDF.type, objective ])

# Subpropiedades de alcance
g.add([melee ,RDF.type, scope ])
g.add([near ,RDF.type, scope ])
g.add([far ,RDF.type, scope ])

# Subpropiedades de velocidad
g.add([slow ,RDF.type, speed ])
g.add([medium ,RDF.type, speed ])
g.add([fast ,RDF.type, speed ])
g.add([veryfast ,RDF.type, speed ])


In [24]:
# Restricciones de Dominio
g.add([timeoflife, RDFS.domain, n.spell ])
g.add([timeoflife, RDFS.domain, n.build ])

In [25]:
# Ingresando los nombres literales de las propiedades y subpropiedades
# Nombres de las clases
g.add([n.card, name , Literal('Carta')])
g.add([n.troop, name , Literal('Tropas')])
g.add([n.build, name , Literal('Estructuras')])
g.add([n.spell, name , Literal('Hechizos')])
g.add([n.charasteristic, name, Literal('Caracteristicas')])

# Nombres de las subclases
g.add([n.giant, name, Literal('Gigante')])
g.add([n.dead, name, Literal('Muerto')])
g.add([n.creature, name, Literal('Criatura')])
g.add([n.dragon, name, Literal('Dragon')])
g.add([n.goblin, name, Literal('Duende')])
g.add([n.magic, name, Literal('Mágico')])
g.add([n.human, name,Literal('Humano')])
g.add([n.hut, name,Literal('Choza')])
g.add([n.tower, name,Literal('Torre')])

# Nombres de la subclase calidad
g.add([rare , name, Literal('Especial')])
g.add([epic , name, Literal('Épica')])
g.add([common , name, Literal('Común') ])
g.add([legendary , name, Literal('Legendaria') ])

# Nombres de la subclase objetivo
g.add([air , name, Literal('Aire') ])
g.add([building , name, Literal('Estructura') ])
g.add([ground , name, Literal('Tierra') ])

# Nombres de la subclase alcance
g.add([melee , name, Literal('Cuerpo a cuerpo') ])
g.add([near , name, Literal('Cerca') ])
g.add([far , name, Literal('Lejos') ])

# Nombres de la subclase velocidad
g.add([slow , name, Literal('Baja')])
g.add([medium , name, Literal('Media') ])
g.add([fast , name, Literal('Alta') ])
g.add([veryfast , name, Literal('Muy Alta') ])

# Nombres de las propiedades
g.add([name , RDFS.label, Literal('Nombre de la carta')])
g.add([level , RDFS.label, Literal('Nivel')])
g.add([quality , RDFS.label, Literal('Calidad')])
g.add([objective , RDFS.label, Literal('Objetivo')])
g.add([scope , RDFS.label, Literal('Alcance')])
g.add([speed , RDFS.label, Literal('Velocidad')])
g.add([radio , RDFS.label, Literal('Radio')])
g.add([timeoflife , RDFS.label, Literal('Tiempo de vida')])
g.add([lifepoints , RDFS.label, Literal('Puntos de vida')])
g.add([hurt , RDFS.label, Literal('Daño')])
g.add([damage , RDFS.label, Literal('Daño por segundo')])
g.add([areadamage , RDFS.label, Literal('Daño de área')])
g.add([elixir , RDFS.label, Literal('elixir')])

# Funciones de transformación

In [26]:
def fun_quality(statement):
    if(statement == 'Especial'):
        return rare
    if(statement == 'Épica'):
        return epic
    if(statement == 'Común'):
        return common
    if(statement == 'Legendaria'):
        return legendary

In [27]:
def fun_objective(diccionario):
    statement = diccionario['objective']
    if(statement == 'De tierra y aire'):
        g.add((URIRef(diccionario['link']), objective, air ) )
        g.add((URIRef(diccionario['link']), objective, ground) )
    if(statement == 'Estructuras'):
        g.add((URIRef(diccionario['link']), objective, building))
    if(statement == 'Terrestres'):
        g.add((URIRef(diccionario['link']), objective, ground ))

In [28]:
def fun_scope(statement):
    if(statement == 'Cuerpo a cuerpo'):
        return melee
    else:
        num = float(statement)
        if(num < 8.0):
            return near
        else:
            return far

In [29]:
def fun_speed(statement):
    if(statement=='Baja'):
        return slow
    if(statement=='Media'):
        return medium
    if(statement=='Alta'):
        return fast
    if(statement=='Muy alta'):
        return veryfast

In [30]:
# funcion que se le pasa por parametro el array de clases, el link y la clase general por si no lo ubica.
def fun_clase(arr_class,link,general_class):
    for clase in arr_class:
        if(clase == 'gigante'):
            g.add([ URIRef(link) ,RDF.type,n.giant])
        if(clase == 'muerto'):
            g.add([ URIRef(link) ,RDF.type,n.dead])
        if(clase == 'criatura'):
            g.add([ URIRef(link) ,RDF.type,n.creature])
        if(clase == 'dragon'):
            g.add([ URIRef(link) ,RDF.type,n.dragon])
        if(clase == 'duende'):
            g.add([ URIRef(link) ,RDF.type,n.goblin])
        if(clase == 'magico'):
            g.add([ URIRef(link) ,RDF.type,n.magic])
        if(clase == 'choza'):
            g.add([ URIRef(link) ,RDF.type,n.hut])
        if(clase == 'torre'):
            g.add([ URIRef(link) ,RDF.type,n.tower])
        if(clase == 'humano'):
            g.add([ URIRef(link) ,RDF.type,n.human])
        else:
            g.add([ URIRef(link) ,RDF.type,general_class])

In [31]:
# Instanciando Cartas Tropas
#lista_diccionario[:71]
for diccionario in lista_diccionario[:71]:
    
    fun_clase(diccionario['classes'],diccionario['link'],n.troop)
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        fun_objective(diccionario)
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    if (diccionario['lifepoints']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))
    if (diccionario['elixir']!= 'None'):
        g.add((URIRef(diccionario['link']), elixir, Literal(diccionario['elixir'])))


In [32]:
# Instanciando Estructuras
#lista_diccionario[71:83]
for diccionario in lista_diccionario[71:83]:
    
    fun_clase(diccionario['classes'],diccionario['link'],n.build)
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        fun_objective(diccionario)
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    if (diccionario['timeoflife']!= 'None'):
        g.add((URIRef(diccionario['link']), timeoflife, Literal(diccionario['timeoflife'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))
    if (diccionario['elixir']!= 'None'):
        g.add((URIRef(diccionario['link']), elixir, Literal(diccionario['elixir'])))


In [33]:
# Instanciando Hechizos
#lista_diccionario[83:]
for diccionario in lista_diccionario[83:]:
   
    fun_clase(diccionario['classes'],diccionario['link'],n.spell)
    g.add((URIRef(diccionario['link']), name, Literal(diccionario['name'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), level, Literal(diccionario['level'])))
    if (diccionario['quality']!= 'None'):
        g.add((URIRef(diccionario['link']), quality, fun_quality(diccionario['quality'])))
    if (diccionario['objective']!= 'None'):
        fun_objective(diccionario)
    if (diccionario['scope']!= 'None'):
        g.add((URIRef(diccionario['link']), scope, fun_scope(diccionario['scope'])))
    if (diccionario['speed']!= 'None'):
        g.add((URIRef(diccionario['link']), speed, fun_speed(diccionario['speed'])))
    if (diccionario['radio']!= 'None'):
        g.add((URIRef(diccionario['link']), radio, Literal(diccionario['radio'])))
    if (diccionario['timeoflife']!= 'None'):
        g.add((URIRef(diccionario['link']), timeoflife, Literal(diccionario['timeoflife'])))
    if (diccionario['level']!= 'None'):
        g.add((URIRef(diccionario['link']), lifepoints, Literal(diccionario['lifepoints'])))
    if (diccionario['hurt']!= 'None'):
        g.add((URIRef(diccionario['link']), hurt, Literal(diccionario['hurt'])))
    if (diccionario['damage']!= 'None'):
        g.add((URIRef(diccionario['link']), damage, Literal(diccionario['damage'])))
    if (diccionario['areadamage']!= 'None'):
        g.add((URIRef(diccionario['link']), areadamage, Literal(diccionario['areadamage'])))
    if (diccionario['elixir']!= 'None'):
        g.add((URIRef(diccionario['link']), elixir, Literal(diccionario['elixir'])))


texto = str(g.serialize(format="n3", encoding='utf-8').decode())
for line in texto.splitlines():
    print(line)

In [34]:
# Exportar el archivo:
g.serialize(destination='statsroyale.rdf', format='xml')

#imprimir tripletas
for s, p, o in g:
    print(s, p, o)

# Inferencias

### Primera inferencia: Encontrando las dependencias superiores de la clase Gigante

In [35]:
clase_base = n.giant
for s in g.transitive_objects(clase_base, RDFS.subClassOf):
    print(g.value(s, name))

Gigante
Tropas
Carta


### Segunda inferencia: Encontrando las dependencias inferiores de la clase Carta 

In [36]:
clase_base = n.card
for s in g.transitive_subjects(RDFS.subClassOf, clase_base):
    print(s,g.value(s, name))

https://statsroyale.com/es/card Carta
https://statsroyale.com/es/build Estructuras
https://statsroyale.com/es/tower Torre
https://statsroyale.com/es/hut Choza
https://statsroyale.com/es/spell Hechizos
https://statsroyale.com/es/troop Tropas
https://statsroyale.com/es/human Humano
https://statsroyale.com/es/dragon Dragon
https://statsroyale.com/es/creature Criatura
https://statsroyale.com/es/goblin Duende
https://statsroyale.com/es/magic Mágico
https://statsroyale.com/es/giant Gigante
https://statsroyale.com/es/dead Muerto


### Tercera inferencia: Encontrando las clases pertenecientes al recurso carta

In [37]:
carta = URIRef(lista_diccionario[1]['link'])
for s in g.transitive_objects(carta, RDF.type):
  print(g.value(s, name))

Gólem
Tropas
Gigante


### Cuarta inferencia: Obteniendo la calidad de una carta

In [38]:
carta = URIRef(lista_diccionario[4]['link'])
for s in g.objects(carta, quality):
    print (s)
    print(g.value(s, name))

https://statsroyale.com/es/epic
Épica


### Quinta inferencia: Obteniendo todos los atributos del recurso carta

In [39]:
carta = URIRef(lista_diccionario[4]['link'])
def atributos(card):
        for s in g.predicate_objects(carta):
                if type(s[1]) == type(Literal("1")):
                    print(g.value(s[0],RDFS.label),s[1])
                else:
                    print(g.value(s[0],RDFS.label),g.value(s[1],name))
                
atributos(carta)


Daño 678
Alcance Cuerpo a cuerpo
Nombre de la carta P.E.K.K.A.
Velocidad Baja
Nivel 9
Puntos de vida 3125
Calidad Épica
Daño por segundo 376
Objetivo Tierra
None Tropas
elixir 7


### Sexta inferencia: Obtener los daños de cada carta

In [40]:
!pip install tabulate

You should consider upgrading via the 'c:\users\jose-\python\python39\python.exe -m pip install --upgrade pip' command.


In [41]:
from tabulate import tabulate
#Daños
tipos = [n.troop,n.build,n.spell]
lista=[[]]
for tipo in tipos:
    for carta in g.subjects(RDF.type,tipo):
        if g.value(carta, damage)!=None:
            lista.append([g.value(carta, name),'Daño',g.value(carta, damage)])
        elif g.value(carta, areadamage)!=None:
            lista.append([g.value(carta, name),'Daño en Área',g.value(carta, areadamage)])
        elif g.value(carta, hurt)!=None:
            lista.append([g.value(carta, name),'Daño por segundo',g.value(carta, hurt)])
print(tabulate(lista,headers=['Nombre de la carta', 'Tipo de daño','Daño']))


Nombre de la carta      Tipo de daño      Daño
----------------------  ----------------  ------------------------------------------------------------
Lanzarrocas             Daño              95
Electrocutadores        Daño              45
Cañón con ruedas        Daño              176
Murciélagos             Daño              51
Gólem de elixir         Daño              162
Espíritu eléctrico      Daño por segundo  82
Mago                    Daño              167
Reclutas reales         Daño              84
Duendes con lanza       Daño              39
Gigante eléctrico       Daño              75
Montapuercos            Daño              165
Esbirros                Daño              84
Chispitas               Daño              275
Dragones esqueleto      Daño              70
P.E.K.K.A.              Daño              376
Bruja nocturna          Daño              173
Bruja madre             Daño              100
Puercos reales          Daño              49
Ariete de batalla       Daño por

### Séptima inferencia: Clasificar por clase tropa, estructura y hechizo

In [42]:
#Clasificar por clase tropas, estructuras y hechizos
tipos = [n.troop,n.build,n.spell]
for tipo in tipos:
    print("...............",(g.value(tipo, name)).upper(),"...............")
    for carta in g.subjects(RDF.type,tipo):
        print(g.value(carta, name))

............... TROPAS ...............
Lanzarrocas
Electrocutadores
Cañón con ruedas
Murciélagos
Gólem de elixir
Espíritu eléctrico
Mago
Reclutas reales
Duendes con lanza
Gigante eléctrico
Montapuercos
Esbirros
Chispitas
Dragones esqueleto
P.E.K.K.A.
Bruja nocturna
Bruja madre
Puercos reales
Ariete de batalla
Bombardero
Verdugo
Arquero mágico
Mago de hielo
Duende gigante
Gigante noble
Pillos
Espíritus de fuego
Duendes
Lanzafuegos
Dragón infernal
Bebé dragón
Megaesbirro
Globo bombástico
Pandilla de duendes
Esqueleto gigante
Duende lanzadardos
Sabueso de lava
Fantasma real
Gigante
Mini P.E.K.K.A.
Horda de esbirros
Gólem de hielo
Mago eléctrico
Barril de esqueletos
Bárbaros
Leñador
Gólem
Rompemuros
Bruja
Dragón eléctrico
Montacarneros
Ejército de esqueletos
Esqueletos
Guardias
Arqueras
Bárbaros de élite
Espíritu de hielo
............... ESTRUCTURAS ...............
Torre infernal
Horno
Choza de bárbaros
Torre bombardera
Ballesta
Choza de duendes
Recolector de elixir
Torre Tesla
Jaula del f

### Manipulación : Subir nivel a una carta

In [147]:
#Subir nivel a una carta
def agregarNivel(x):
    nivelX=0
    for s,d,o in g.triples((x,RDF.type,None)):
        nameX=g.value(s,name)
        nivelX=int(g.value(s,level))
        print('Nivel actual',nivelX)
        nivelX+=1
        g.set([x, level,Literal(nivelX)])
        print('Se ha subido al nivel',nivelX)
    return nivelX

In [148]:
#Prueba de subir nivel
agregarNivel(URIRef("https://statsroyale.com/es/card/Musketeer"))
print('nivel de la carta actualizada:',g.value(URIRef("https://statsroyale.com/es/card/Musketeer"),level))

Nivel actual 10
Se ha subido al nivel 11
nivel de la carta actualizada: 11


### Combate
La función combate necesita de dos cartas y sus respectivas vidas actuales, encuentra el nombre y daño de cada carta y las hace pelear, la carta gana según su daño y vida, una vez obtenido el ganador se devuelve 1 si es la primera carta o 2 si es la segunda, además también devuelve la vida actual del ganador

#### Obtener elixir de una carta
El elixir permite poner cartas en el campo batalla, cada carta posee una cantidad de elixir necesaria para ser utilizada.

In [152]:
def elixir(x):
    elixirX=0
    for s,d,o in g.triples((x,RDF.type,None)):
        elixirX=int(g.value(s,n.elixir))
    return elixirX

In [153]:
iceGolem=URIRef("https://statsroyale.com/es/card/Ice+Golem")
print('Elixir de',g.value(iceGolem,name) ,': ',elixir(iceGolem))

Elixir de Gólem de hielo :  2


#### Obtener daño de una carta


In [156]:
def daño(x):
    dañoX=0.0
    for s,d,o in g.triples((x,RDF.type,None)):
        if g.value(s,damage)!=None:
            dañoX=float(g.value(s,damage))
        elif g.value(s,areadamage)!=None:
            dañoX=float(g.value(s,areadamage))
        elif g.value(s,hurt)!=None:
            dañoX=float(g.value(s,hurt))
    return dañoX

In [157]:
musketer=URIRef("https://statsroyale.com/es/card/Musketeer")
print('Daño: ',daño(musketer))

Daño:  164.0


#### Obtener vida de una carta

In [139]:
def vida(x):
    vidaX=0.0
    for s,d,o in g.triples((x,RDF.type,None)):
        vidaX=float(g.value(s,lifepoints))
    return vidaX

In [158]:
#ejemplo
musketer=URIRef("https://statsroyale.com/es/card/Musketeer")
print('Vida: ',vida(musketer))

Vida:  598.0


#### Obtener nombre de una carta

In [141]:
def nombre(x):
    nameX=''
    for s,d,o in g.triples((x,RDF.type,None)):
        nameX=str(g.value(s,name))
    return nameX

In [142]:
#ejemplo
musketer=URIRef("https://statsroyale.com/es/card/Musketeer")
print('Nombre: ',nombre(musketer))

Nombre:  Mosquetera


#### Función combate

In [161]:
#funcion combate
def combat(x,lifeX,y,lifeY):
    for s,d,o in g.triples((x,RDF.type,None)):
        nameX=str(g.value(s,name))
        damX=daño(x)
    for s,d,o in g.triples((y,RDF.type,None)):
        nameY=str(g.value(s,name))
        damY=daño(y)
    print('Comienza el combate entre ',nameX,' y ',nameY)
    print('Vida de ',nameX,' : ',lifeX,', Daño: ',damX)
    print('Vida de ',nameY,' : ',lifeY,', Daño: ',damY)
    while lifeX>0 and lifeY>0: 
        lifeX-=damY
        lifeY-=damX
    if lifeX<=0  and lifeY<=0:
        lifeX=0
        lifeY=0
        print('Empate!, ambos tienen 0 de vida')
        return [2,lifeY]
    elif lifeY<=0:
        lifeY=0
        print('Carta de jugador 1 ha ganado,',nameX,' ha ganado!, le queda', lifeX,' de vida')
        return [1,lifeX]
    elif lifeX<=0:
        lifeX=0
        print('Carta de jugador 2 ha ganado, ',nameY,' ha ganado!, le queda', lifeY,' de vida')
        return [2,lifeY]

In [162]:
#prueba de combate
musketer=URIRef("https://statsroyale.com/es/card/Musketeer")
GiantSkeleton=URIRef("https://statsroyale.com/es/card/Dark+Prince")

Ganador,lifeGanador=combat(musketer,400,GiantSkeleton,600)
print(Ganador)
print(lifeGanador)

Comienza el combate entre  Mosquetera  y  Príncipe oscuro
Vida de  Mosquetera  :  400 , Daño:  164.0
Vida de  Príncipe oscuro  :  600 , Daño:  158.0
Carta de jugador 2 ha ganado,  Príncipe oscuro  ha ganado!, le queda 108.0  de vida
2
108.0


### Simulación del juego
Se crea una simulación usando las inferencias anteriores. Se define 5 cartas, luego se guardan en cartas1 y cartas2 q vienen a ser la mano de cartas de cada jugador, además se obtienen la vida de cada carta usando la inferencia vida. Se pide escoger entre:
Opción 1: Mostrar características de las cartas del jugador 1. 
Opción 2: Mostrar las características de las cartas del jugador 2.
Opción 3, combatir.
Al combatir se pide ingresar primero el nombre de la carta del jugador 1 y luego del jugador 2(el nombre de la carta debe ser escrita igual que la mostrada al escoger las opciones 1 o 2, mayúsculas, tilde, etc), una vez que los pasos son correctos el combate empieza, se puede seguir jugando hasta que se escoja la opción salir o hasta que todas las cartas de alguno de los jugadores tengan 0 de vida.

In [166]:
#Defino las cartas
musketer=URIRef("https://statsroyale.com/es/card/Musketeer")
giantSkeleton=URIRef("https://statsroyale.com/es/card/Dark+Prince")
giantGoblin=URIRef("https://statsroyale.com/es/card/Goblin+Giant")
valquiria=URIRef("https://statsroyale.com/es/card/Valkyrie")
miner=URIRef("https://statsroyale.com/es/card/Miner")
#cartas de jugador 1
cartas1=[miner,giantSkeleton,giantGoblin,valquiria]
vidas1=[vida(miner),vida(giantSkeleton),vida(giantGoblin),vida(valquiria)]
#cartas de jugador 2
cartas2=[musketer,giantSkeleton,giantGoblin,valquiria]
vidas2=[vida(musketer),vida(giantSkeleton),vida(giantGoblin),vida(valquiria)]

#Verifica si la vida de las cartas están en 0
def verify(vidas):
    for i in vidas:
        if(i>0):
            return True
    return False

#Compara el nombre ingresado con el nombre de la carta y verifica si encuentra y obtiene su posicion en el arreglo.
def lugar(nom,cartas):
    for count,value in enumerate(cartas):
        if nombre(value)==nom:
            print('Se encontró la unidad')
            return count
    print('No se encontró la unidad')
    return -1

#Contador par máximos movimientos
k=0
#Juego
while verify(vidas1) and verify(vidas2):
    k+=1
    if k>1000:
        break

    decision=input('¿Desea seguir jugando?/(Cualquier tecla=Si/N=No)')
    if decision=='N' or decision=='n':
        break
    
    opcion=input('Opciones\n1=Características de cartas de jugador 1\n2=Características de cartas de jugador 2\n3=Combatir!')
    if opcion=='1':
        #Muestra las cartas del jugador 1
        print('CARTAS DE JUGADOR 1:')
        for count,value in enumerate(cartas1):
            print('Nombre: ',nombre(value),', vida: ',vidas1[count],', daño: ',daño(value), ', elixir: ',elixir(cartas1[count]))
    elif opcion=='2':
        #Muestra las cartas del jugador 2
        print('CARTAS DE JUGADOR 2:')
        for count,value in enumerate(cartas2):
            print('Nombre: ',nombre(value),', vida: ',vidas2[count],', daño: ',daño(value), ', elixir: ',elixir(cartas2[count]))
    elif opcion=='3':
        #Combatir cartas
        print('INICIA EL COMBATE!:')
        aliado=input('Ingrese nombre de la carta del jugador 1')
        enemigo=''
        posA=-1
        posE=-1
        #Encuentra la posición de la carta del jugador 1
        posA=lugar(aliado,cartas1)
        if posA>=0:
            if vidas1[posA]==vida(cartas1[posA]):
                print('Colocando carta ',nombre(cartas1[posA]),' se ha gastado ',elixir(cartas1[posA]),' de elixir')
            enemigo=input('Ingrese nombre de la carta del jugador 2')
            #Encuentra la posición de la carta del jugador 2
            posE=lugar(enemigo,cartas2)
            if posE>=0:
                if vidas2[posE]==vida(cartas2[posE]):
                    print('Colocando carta ',nombre(cartas2[posE]),' se ha gastado ',elixir(cartas2[posE]),' de elixir')
                #Se obtiene el ganador y la vida con la que queda después del combate
                Gan,vidaGan=combat(cartas1[posA],vidas1[posA],cartas2[posE],vidas2[posE])
                #Si gana el jugador 1 se guarda la vida correspondiente de cada carta
                if 1==Gan:
                    vidas1[posA]=vidaGan
                    vidas2[posE]=0
                #Si gana el jugador 2 se guarda la vida correspondiente de cada carta
                elif 2==Gan:
                    vidas2[posE]=vidaGan
                    vidas1[posA]=0
print('LA PARTIDA HA TERMINADO')
#Se verifica si alguien ha ganado el juego o si ha sido empate
if verify(vidas1)==False and verify(vidas2)==False:
    print('Ambos Jugadores han quedado empate!')
elif verify(vidas1):
    print('Jugador 1 ha ganado la partida!')
elif verify(vidas2):
    print('Jugador 1 ha ganado la partida!')

CARTAS DE JUGADOR 1:
Nombre:  Minero , vida:  1000.0 , daño:  133.0 , elixir:  3
Nombre:  Príncipe oscuro , vida:  1030.0 , daño:  158.0 , elixir:  4
Nombre:  Duende gigante , vida:  2616.0 , daño:  97.0 , elixir:  6
Nombre:  Valquiria , vida:  1654.0 , daño:  147.0 , elixir:  4
CARTAS DE JUGADOR 2:
Nombre:  Mosquetera , vida:  598.0 , daño:  164.0 , elixir:  4
Nombre:  Príncipe oscuro , vida:  1030.0 , daño:  158.0 , elixir:  4
Nombre:  Duende gigante , vida:  2616.0 , daño:  97.0 , elixir:  6
Nombre:  Valquiria , vida:  1654.0 , daño:  147.0 , elixir:  4
INICIA EL COMBATE!:
Se encontró la unidad
Colocando carta  Minero  se ha gastado  3  de elixir
Se encontró la unidad
Colocando carta  Mosquetera  se ha gastado  4  de elixir
Comienza el combate entre  Minero  y  Mosquetera
Vida de  Minero  :  1000.0 , Daño:  133.0
Vida de  Mosquetera  :  598.0 , Daño:  164.0
Carta de jugador 1 ha ganado, Minero  ha ganado!, le queda 180.0  de vida
INICIA EL COMBATE!:
Se encontró la unidad
Colocando c